!pip install tweepy==3.10.0
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi

In [1]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tasya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Read data
import pandas as pd
def load_data():
    data = pd.read_csv('translate_cashless.csv')
    return data

In [4]:
tweet_df = load_data()

In [5]:
tweet_df.to_csv('translate_cashless.csv', encoding='utf8', index = False)

In [6]:
df = pd.read_csv('translate_cashless.csv')
df.head(10)

,Tweet
0,I want to remove ordinary cashless banking
1,it's rarely cashless when you ride a motorcycl...
2,Cashless payments are not evenly distributed a...
3,the seller wants to know how to use cashless
4,cashless does not hold money
5,nice to pay cashless
6,implement cashless
7,likes cashless lifestyle premium card holder r...
8,Now peekaboo sellers around if they can pay us...
9,informed to all taxpayers to make cashless lan...


In [7]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    return text

In [8]:
# Memasukkan data
data = pd.read_csv("translate_cashless.csv", header=0)
data.head()

,Tweet
0,I want to remove ordinary cashless banking
1,it's rarely cashless when you ride a motorcycl...
2,Cashless payments are not evenly distributed a...
3,the seller wants to know how to use cashless
4,cashless does not hold money


In [9]:
# Case folding
def clean_lower(lwr):
 lwr = lwr.lower() # lowercase text
 return lwr

In [10]:
# Membuat kolom tambahan untuk data yang telah di casefolding
data['twt'] = data['Tweet'].apply(clean_lower)
data['twt']

0             i want to remove ordinary cashless banking
1      it's rarely cashless when you ride a motorcycl...
2      cashless payments are not evenly distributed a...
3           the seller wants to know how to use cashless
4                           cashless does not hold money
                             ...                        
590                                cashless pay by swipe
591                             ica food is cashless now
592    cashless is all good until your banking error ...
593    a cashless type wallet really has to carry a w...
594                                  cardholder cashless
Name: twt, Length: 595, dtype: object

In [11]:
#Membuat dataframe
casefolding=pd.DataFrame(data['twt'])
casefolding

,twt
0,i want to remove ordinary cashless banking
1,it's rarely cashless when you ride a motorcycl...
2,cashless payments are not evenly distributed a...
3,the seller wants to know how to use cashless
4,cashless does not hold money
...,...
590,cashless pay by swipe
591,ica food is cashless now
592,cashless is all good until your banking error ...
593,a cashless type wallet really has to carry a w...


In [12]:
# Menyimpan data hasil casefolding ke excel
file_name = 'casefolding.xlsx'
casefolding.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [13]:
# Remove Punctuation
84
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def remove_punct(text):
 text = clean_spcl.sub('', text)
 text = clean_symbol.sub(' ', text)
 return text

In [14]:
# Remove number
data['remove_punct']=data['twt'].str.replace(r'[\d+]', '')

C:\Users\Tasya\AppData\Local\Temp\ipykernel_7616\747957790.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['remove_punct']=data['twt'].str.replace(r'[\d+]', '')


In [15]:
# Membuat kolom tambahan untuk data description yang telah diremovepunctuation
data['remove_punct'] = data['twt'].apply(remove_punct)
data['remove_punct']

0             i want to remove ordinary cashless banking
1      it s rarely cashless when you ride a motorcycl...
2      cashless payments are not evenly distributed a...
3           the seller wants to know how to use cashless
4                           cashless does not hold money
                             ...                        
590                                cashless pay by swipe
591                             ica food is cashless now
592    cashless is all good until your banking error ...
593    a cashless type wallet really has to carry a w...
594                                  cardholder cashless
Name: remove_punct, Length: 595, dtype: object

In [16]:
# Remove Double Whitespace
def _normalize_whitespace(text):
 """
 This function normalizes whitespaces, removing duplicates.
 """
 corrected = str(text)
 corrected = re.sub(r"//t",r"\t", corrected)
 corrected = re.sub(r"( )\1+",r"\1", corrected)
 corrected = re.sub(r"(\n)\1+",r"\1", corrected)
 corrected = re.sub(r"(\r)\1+",r"\1", corrected)
 corrected = re.sub(r"(\t)\1+",r"\1", corrected)
 return corrected.strip(" ")
data['remove_double_ws'] = data['remove_punct'].apply(_normalize_whitespace)
data['remove_double_ws']

0             i want to remove ordinary cashless banking
1      it s rarely cashless when you ride a motorcycl...
2      cashless payments are not evenly distributed a...
3           the seller wants to know how to use cashless
4                           cashless does not hold money
                             ...                        
590                                cashless pay by swipe
591                             ica food is cashless now
592    cashless is all good until your banking error ...
593    a cashless type wallet really has to carry a w...
594                                  cardholder cashless
Name: remove_double_ws, Length: 595, dtype: object

In [18]:
# Menyimpan hasil remove punctuation ke excel
removepunctuation=pd.DataFrame(data['remove_double_ws'])
file_name = 'cleaning.xlsx'
removepunctuation.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [19]:
# Memasukkan Stopword removal Bahasa Inggris
stopword=set(stopwords.words('english'))
sw=pd.DataFrame(stopword)
sw

,0
0,some
1,more
2,just
3,did
4,below
...,...
174,to
175,s
176,you're
177,myself


In [188]:
# Mengunduh stopword dan menyimpannya ke excel
file_name = 'stopwords.xlsx'
sw.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [20]:
# Memasukkan stopword
stw = open("stopwords.txt")
# Use this to read file content as a stream:
line = stw.read()
stopword = line.split()
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

In [21]:
# Membuat kolom tambahan untuk data yang telah distopwordsremoval
data['remove_sw'] = data['remove_double_ws'].apply(clean_stopwords)
data['remove_sw']

0                  want remove ordinary cashless banking
1                   rarely cashless ride motorcycle taxi
2      cashless payments evenly distributed gas stations
3                                    seller use cashless
4                                    cashless hold money
                             ...                        
590                                   cashless pay swipe
591                                    ica food cashless
592                   cashless good banking error paying
593    cashless type wallet really carry wallet bring...
594                                  cardholder cashless
Name: remove_sw, Length: 595, dtype: object

In [22]:
# Memanggil stopword
stopword

['0',
 '0',
 "didn't",
 '1',
 "aren't",
 '2',
 'o',
 '3',
 'can',
 '4',
 'my',
 '5',
 "weren't",
 '6',
 'were',
 '7',
 'those',
 '8',
 'had',
 '9',
 "needn't",
 '10',
 'am',
 '11',
 'your',
 '12',
 'd',
 '13',
 "mustn't",
 '14',
 'she',
 '15',
 'this',
 '16',
 'not',
 '17',
 'further',
 '18',
 'up',
 '19',
 'hers',
 '20',
 'than',
 '21',
 'to',
 '22',
 'does',
 '23',
 'about',
 '24',
 'was',
 '25',
 "should've",
 '26',
 "it's",
 '27',
 'doing',
 '28',
 'theirs',
 '29',
 'until',
 '30',
 'himself',
 '31',
 'with',
 '32',
 "you'd",
 '33',
 're',
 '34',
 'any',
 '35',
 'have',
 '36',
 'ourselves',
 '37',
 'having',
 '38',
 'before',
 '39',
 'because',
 '40',
 'in',
 '41',
 'ours',
 '42',
 'some',
 '43',
 't',
 '44',
 'isn',
 '45',
 'same',
 '46',
 "mightn't",
 '47',
 "shan't",
 '48',
 'be',
 '49',
 'he',
 '50',
 'their',
 '51',
 'ain',
 '52',
 'no',
 '53',
 "doesn't",
 '54',
 'for',
 '55',
 'own',
 '56',
 'ma',
 '57',
 "she's",
 '58',
 'shouldn',
 '59',
 'being',
 '60',
 'him',
 '61',
 'h

In [23]:
# Membuat stopword tambahan
tambahan = pd.DataFrame(data['remove_sw'])
tambahan
data['tambah_swr']= tambahan.replace(to_replace
=['also','always','another','look','enough',
'gojek','gofood','grabfood','shopeefood','grab','emot','emoticon','emoticons',
'indonesia','jawa','east','maybe','keeping','guys','jakarta','kalimantan',
'please','rather','branch','hope','briefing','tomorrow','btw',
'by the way','sometime','sometimes','bring','pretty','survey',
'direction','thousand','finish','alhamdulillah','soon',
'land','won','into','many','isnt','note','notes',
'need','under','branches','provide','overall','gather',
'look','buy','arrive','arrived','hasnt','keep','nonac',
'must','was','stop','came','back','definitely','visit', 'buy',
'section','half','almost','near','between','future','bought',
'front','first','later','people','oclock','hour','hours',
'minutes','according','causing','problem','problems',
'road','surveyed','turn','since','asked','leave',
'beside','besides','yesterday','until','use','happens',
'runs','opens','yet','never','thats','come','improved','extra',
'chat','whether','super','shared','being','having','wont',
'buyer','enough','think','may','turned','wear','after',
'say','new','improve','does','would','your','above',
'just','them','left','own','attached',
'taking','saving','needs','by','selling','choice','peak','himself',
'accumulation','aacumulations','overall','southeast','around',
'theres','ago','draw','area','burger','pizza','brimo','flazz','bri'],value ="", regex= True)
data['tambah_swr']

0                  want remove ordinary cashless banking
1                   rarely cashless ride motorcycle taxi
2      cashless payments evenly distributed gas stations
3                                       seller  cashless
4                                    cashless hold money
                             ...                        
590                                   cashless pay swipe
591                                    ica food cashless
592                   cashless good banking error paying
593    cashless type wallet really carry wallet  mone...
594                                  cardholder cashless
Name: tambah_swr, Length: 595, dtype: object

In [24]:
# Menyimpan data yang sudah dilakukan penghapusan stopword
clean_stopwordsrev=pd.DataFrame(data['tambah_swr'])
file_name = 'filtering.xlsx'
clean_stopwordsrev.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [25]:
# Mengunduh package wordnet
nltk.download('wordnet')
wn= nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tasya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Tasya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [27]:
# Melakukan lemmatization
def lemmatization(text):
 text = ' '.join(wn.lemmatize(word) for word in text.split() if
word in text)
 return text

In [28]:
# Buat kolom tambahan untuk data description yang telah dilemmatization
data['twt_remove_lemma'] = data['tambah_swr'].apply(lemmatization)
data['twt_remove_lemma']
data['twt_removefix']= data['twt_remove_lemma']

In [29]:
# Menyimpan data hasil lemmatization ke excel
lemmatization=pd.DataFrame(data['twt_remove_lemma'])
file_name = 'lemmatization.xlsx'

# saving the excel
lemmatization.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [30]:
# Melakukan tokenizing
import re
# Function to Tokenize words
def tokenize(text):
    tokens = re.split('\W+', text) #W+ means that either a word character (A-Za-z0-9_) or a dash (-) can go there.
    return tokens

data['twt_tokenize'] = data['twt_remove_lemma'].apply(lambda x:tokenize(x.lower()))
data['twt_tokenize']

0            [want, remove, ordinary, cashless, banking]
1             [rarely, cashless, ride, motorcycle, taxi]
2      [cashless, payment, evenly, distributed, gas, ...
3                                     [seller, cashless]
4                                [cashless, hold, money]
                             ...                        
590                               [cashless, pay, swipe]
591                                [ica, food, cashless]
592             [cashless, good, banking, error, paying]
593    [cashless, type, wallet, really, carry, wallet...
594                               [cardholder, cashless]
Name: twt_tokenize, Length: 595, dtype: object

In [31]:
# Menyimpan hasil data tokenizing ke excel
tokenizing=pd.DataFrame(data['twt_tokenize'])
file_name = 'tokenizing.xlsx'

# saving the excel
tokenizing.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [32]:
# Membuka data hasil tokenizing
import pandas as pd
def load_data():
 data=pd.read_excel('tokenizing.xlsx')
 return data
data = load_data()
data.head()

,Unnamed: 0,twt_tokenize
0,0,"['want', 'remove', 'ordinary', 'cashless', 'ba..."
1,1,"['rarely', 'cashless', 'ride', 'motorcycle', '..."
2,2,"['cashless', 'payment', 'evenly', 'distributed..."
3,3,"['seller', 'cashless']"
4,4,"['cashless', 'hold', 'money']"


In [202]:
# Menyimpan ke format csv
data.to_csv('tokenizing.csv',encoding='utf8', index=False)
import pandas as pd
def load_data():
 data=pd.read_csv('tokenizing.csv')
 return data
data = load_data()
data.head()

,Unnamed: 0,twt_tokenize
0,0,"['want', 'remove', 'ordinary', 'cashless', 'ba..."
1,1,"['rarely', 'cashless', 'ride', 'motorcycle', '..."
2,2,"['cashless', 'payment', 'evenly', 'distributed..."
3,3,"['seller', 'cashless']"
4,4,"['cashless', 'hold', 'money']"


In [33]:
# Mengubah data menjadi bentuk string
data['twt_tokenize']=data['twt_tokenize'].apply(str)

In [34]:
# Melakukan preprocessing ulang
# remove number
def remove_number(text):
    return re.sub(r"\d+", "", text)
data['twt_removenum'] = data['twt_tokenize'].apply(remove_number)
# remove punctuation
import string
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))
data['twt_removepunc'] = data['twt_removenum'].apply(remove_punctuation)
# remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()
data['twt_whtspc'] = data['twt_removepunc'].apply(remove_whitespace_LT)
# remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)
data['twt_multiplew'] = data['twt_whtspc'].apply(remove_whitespace_multiple)
# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)
data['tweet'] = data['twt_multiplew'].apply(remove_singl_char)
data['tweet']

0                  want remove ordinary cashless banking
1                   rarely cashless ride motorcycle taxi
2        cashless payment evenly distributed gas station
3                                        seller cashless
4                                    cashless hold money
                             ...                        
590                                   cashless pay swipe
591                                    ica food cashless
592                   cashless good banking error paying
593    cashless type wallet really carry wallet money...
594                                  cardholder cashless
Name: tweet, Length: 595, dtype: object

In [35]:
# remove tweet kosong
data= data[data['tweet']!='']
data.head(5)

,Unnamed: 0,twt_tokenize,twt_removenum,twt_removepunc,twt_whtspc,twt_multiplew,tweet
0,0,"['want', 'remove', 'ordinary', 'cashless', 'ba...","['want', 'remove', 'ordinary', 'cashless', 'ba...",want remove ordinary cashless banking,want remove ordinary cashless banking,want remove ordinary cashless banking,want remove ordinary cashless banking
1,1,"['rarely', 'cashless', 'ride', 'motorcycle', '...","['rarely', 'cashless', 'ride', 'motorcycle', '...",rarely cashless ride motorcycle taxi,rarely cashless ride motorcycle taxi,rarely cashless ride motorcycle taxi,rarely cashless ride motorcycle taxi
2,2,"['cashless', 'payment', 'evenly', 'distributed...","['cashless', 'payment', 'evenly', 'distributed...",cashless payment evenly distributed gas station,cashless payment evenly distributed gas station,cashless payment evenly distributed gas station,cashless payment evenly distributed gas station
3,3,"['seller', 'cashless']","['seller', 'cashless']",seller cashless,seller cashless,seller cashless,seller cashless
4,4,"['cashless', 'hold', 'money']","['cashless', 'hold', 'money']",cashless hold money,cashless hold money,cashless hold money,cashless hold money


In [36]:
# reset index
data = data.reset_index(drop=True)
data.head(5)

,Unnamed: 0,twt_tokenize,twt_removenum,twt_removepunc,twt_whtspc,twt_multiplew,tweet
0,0,"['want', 'remove', 'ordinary', 'cashless', 'ba...","['want', 'remove', 'ordinary', 'cashless', 'ba...",want remove ordinary cashless banking,want remove ordinary cashless banking,want remove ordinary cashless banking,want remove ordinary cashless banking
1,1,"['rarely', 'cashless', 'ride', 'motorcycle', '...","['rarely', 'cashless', 'ride', 'motorcycle', '...",rarely cashless ride motorcycle taxi,rarely cashless ride motorcycle taxi,rarely cashless ride motorcycle taxi,rarely cashless ride motorcycle taxi
2,2,"['cashless', 'payment', 'evenly', 'distributed...","['cashless', 'payment', 'evenly', 'distributed...",cashless payment evenly distributed gas station,cashless payment evenly distributed gas station,cashless payment evenly distributed gas station,cashless payment evenly distributed gas station
3,3,"['seller', 'cashless']","['seller', 'cashless']",seller cashless,seller cashless,seller cashless,seller cashless
4,4,"['cashless', 'hold', 'money']","['cashless', 'hold', 'money']",cashless hold money,cashless hold money,cashless hold money,cashless hold money


In [37]:
# Menyimpan data clean ke excel
data=data["tweet"]
dataclean=pd.DataFrame(data)
file_name = 'clean_cashless.xlsx'
dataclean.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [38]:
# Menyimpan data clean ke csv
dataclean.to_csv('clean_cashless.csv',encoding='utf8', index=False)
print('DataFrame is written to CSV File successfully.')

DataFrame is written to CSV File successfully.
